In [1]:
import numpy as np
import random, math
import copy

Dimension = 4
qubits = 3
A = np.array([[-9, -2,-2,9], [-3, 10,-3,-3], [-8,6,-9,-8], [-8,8,10,9]]) # Row vectors
b = np.array([-29, 19, 30, -19]) # Column vectors

# 24 qubits with integer solutions
# x = q1 + 2q2 + 4q3 - q4 - 2q5 - 4q6
# x = {{0}, {1}, {0}, {-3}}
# mininum -2463

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
                
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef

# Print Matrix Q|
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = i + 1
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = i + 1
            qdrt2 = j + 1
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")      

# Matrix Q is
[[  -14.   872.  1744.     0.     0.     0.  -248.  -496.  -992.   248.
    496.   992.    38.    76.   152.   -38.   -76.  -152.  -160.  -320.
   -640.   160.   320.   640.]
 [    0.   408.  3488.     0.     0.     0.  -496.  -992. -1984.   496.
    992.  1984.    76.   152.   304.   -76.  -152.  -304.  -320.  -640.
  -1280.   320.   640.  1280.]
 [    0.     0.  2560.     0.     0.     0.  -992. -1984. -3968.   992.
   1984.  3968.   152.   304.   608.  -152.  -304.  -608.  -640. -1280.
  -2560.   640.  1280.  2560.]
 [    0.     0.     0.   450.   872.  1744.   248.   496.   992.  -248.
   -496.  -992.   -38.   -76.  -152.    38.    76.   152.   160.   320.
    640.  -160.  -320.  -640.]
 [    0.     0.     0.     0.  1336.  3488.   496.   992.  1984.  -496.
   -992. -1984.   -76.  -152.  -304.    76.   152.   304.   320.   640.
   1280.  -320.  -640. -1280.]
 [    0.     0.     0.     0.     0.  4416.   992.  1984.  3968.  -992.
  -1984. -3968.  -152.  -304.  -608.   

In [2]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-14.0, ('q2','q2'):408.0, ('q3','q3'):2560.0, ('q4','q4'):450.0, ('q5','q5'):1336.0, ('q6','q6'):4416.0, ('q7','q7'):-348.0, ('q8','q8'):-288.0, ('q9','q9'):1056.0, ('q10','q10'):756.0, ('q11','q11'):1920.0, ('q12','q12'):5472.0, ('q13','q13'):1112.0, ('q14','q14'):2612.0, ('q15','q15'):6776.0, ('q16','q16'):-724.0, ('q17','q17'):-1060.0, ('q18','q18'):-568.0, ('q19','q19'):1693.0, ('q20','q20'):3856.0, ('q21','q21'):9592.0, ('q22','q22'):-1223.0, ('q23','q23'):-1976.0, ('q24','q24'):-2072.0}

quadratic = {('q1','q2'):872.0, ('q1','q3'):1744.0, ('q1','q7'):-248.0, ('q1','q8'):-496.0, ('q1','q9'):-992.0, ('q1','q10'):248.0, ('q1','q11'):496.0, ('q1','q12'):992.0, ('q1','q13'):38.0, ('q1','q14'):76.0, ('q1','q15'):152.0, ('q1','q16'):-38.0, ('q1','q17'):-76.0, ('q1','q18'):-152.0, ('q1','q19'):-160.0, ('q1','q20'):-320.0, ('q1','q21'):-640.0, ('q1','q22'):160.0, ('q1','q23'):320.0, ('q1','q24'):640.0, ('q2','q3'):3488.0, ('q2','q7'):-496.0, ('q2','q8'):-992.0, ('q2','q9'):-1984.0, ('q2','q10'):496.0, ('q2','q11'):992.0, ('q2','q12'):1984.0, ('q2','q13'):76.0, ('q2','q14'):152.0, ('q2','q15'):304.0, ('q2','q16'):-76.0, ('q2','q17'):-152.0, ('q2','q18'):-304.0, ('q2','q19'):-320.0, ('q2','q20'):-640.0, ('q2','q21'):-1280.0, ('q2','q22'):320.0, ('q2','q23'):640.0, ('q2','q24'):1280.0, ('q3','q7'):-992.0, ('q3','q8'):-1984.0, ('q3','q9'):-3968.0, ('q3','q10'):992.0, ('q3','q11'):1984.0, ('q3','q12'):3968.0, ('q3','q13'):152.0, ('q3','q14'):304.0, ('q3','q15'):608.0, ('q3','q16'):-152.0, ('q3','q17'):-304.0, ('q3','q18'):-608.0, ('q3','q19'):-640.0, ('q3','q20'):-1280.0, ('q3','q21'):-2560.0, ('q3','q22'):640.0, ('q3','q23'):1280.0, ('q3','q24'):2560.0, ('q4','q5'):872.0, ('q4','q6'):1744.0, ('q4','q7'):248.0, ('q4','q8'):496.0, ('q4','q9'):992.0, ('q4','q10'):-248.0, ('q4','q11'):-496.0, ('q4','q12'):-992.0, ('q4','q13'):-38.0, ('q4','q14'):-76.0, ('q4','q15'):-152.0, ('q4','q16'):38.0, ('q4','q17'):76.0, ('q4','q18'):152.0, ('q4','q19'):160.0, ('q4','q20'):320.0, ('q4','q21'):640.0, ('q4','q22'):-160.0, ('q4','q23'):-320.0, ('q4','q24'):-640.0, ('q5','q6'):3488.0, ('q5','q7'):496.0, ('q5','q8'):992.0, ('q5','q9'):1984.0, ('q5','q10'):-496.0, ('q5','q11'):-992.0, ('q5','q12'):-1984.0, ('q5','q13'):-76.0, ('q5','q14'):-152.0, ('q5','q15'):-304.0, ('q5','q16'):76.0, ('q5','q17'):152.0, ('q5','q18'):304.0, ('q5','q19'):320.0, ('q5','q20'):640.0, ('q5','q21'):1280.0, ('q5','q22'):-320.0, ('q5','q23'):-640.0, ('q5','q24'):-1280.0, ('q6','q7'):992.0, ('q6','q8'):1984.0, ('q6','q9'):3968.0, ('q6','q10'):-992.0, ('q6','q11'):-1984.0, ('q6','q12'):-3968.0, ('q6','q13'):-152.0, ('q6','q14'):-304.0, ('q6','q15'):-608.0, ('q6','q16'):152.0, ('q6','q17'):304.0, ('q6','q18'):608.0, ('q6','q19'):640.0, ('q6','q20'):1280.0, ('q6','q21'):2560.0, ('q6','q22'):-640.0, ('q6','q23'):-1280.0, ('q6','q24'):-2560.0, ('q7','q8'):816.0, ('q7','q9'):1632.0, ('q7','q19'):-48.0, ('q7','q20'):-96.0, ('q7','q21'):-192.0, ('q7','q22'):48.0, ('q7','q23'):96.0, ('q7','q24'):192.0, ('q8','q9'):3264.0, ('q8','q19'):-96.0, ('q8','q20'):-192.0, ('q8','q21'):-384.0, ('q8','q22'):96.0, ('q8','q23'):192.0, ('q8','q24'):384.0, ('q9','q19'):-192.0, ('q9','q20'):-384.0, ('q9','q21'):-768.0, ('q9','q22'):192.0, ('q9','q23'):384.0, ('q9','q24'):768.0, ('q10','q11'):816.0, ('q10','q12'):1632.0, ('q10','q19'):48.0, ('q10','q20'):96.0, ('q10','q21'):192.0, ('q10','q22'):-48.0, ('q10','q23'):-96.0, ('q10','q24'):-192.0, ('q11','q12'):3264.0, ('q11','q19'):96.0, ('q11','q20'):192.0, ('q11','q21'):384.0, ('q11','q22'):-96.0, ('q11','q23'):-192.0, ('q11','q24'):-384.0, ('q12','q19'):192.0, ('q12','q20'):384.0, ('q12','q21'):768.0, ('q12','q22'):-192.0, ('q12','q23'):-384.0, ('q12','q24'):-768.0, ('q13','q14'):776.0, ('q13','q15'):1552.0, ('q13','q19'):306.0, ('q13','q20'):612.0, ('q13','q21'):1224.0, ('q13','q22'):-306.0, ('q13','q23'):-612.0, ('q13','q24'):-1224.0, ('q14','q15'):3104.0, ('q14','q19'):612.0, ('q14','q20'):1224.0, ('q14','q21'):2448.0, ('q14','q22'):-612.0, ('q14','q23'):-1224.0, ('q14','q24'):-2448.0, ('q15','q19'):1224.0, ('q15','q20'):2448.0, ('q15','q21'):4896.0, ('q15','q22'):-1224.0, ('q15','q23'):-2448.0, ('q15','q24'):-4896.0, ('q16','q17'):776.0, ('q16','q18'):1552.0, ('q16','q19'):-306.0, ('q16','q20'):-612.0, ('q16','q21'):-1224.0, ('q16','q22'):306.0, ('q16','q23'):612.0, ('q16','q24'):1224.0, ('q17','q18'):3104.0, ('q17','q19'):-612.0, ('q17','q20'):-1224.0, ('q17','q21'):-2448.0, ('q17','q22'):612.0, ('q17','q23'):1224.0, ('q17','q24'):2448.0, ('q18','q19'):-1224.0, ('q18','q20'):-2448.0, ('q18','q21'):-4896.0, ('q18','q22'):1224.0, ('q18','q23'):2448.0, ('q18','q24'):4896.0, ('q19','q20'):940.0, ('q19','q21'):1880.0, ('q20','q21'):3760.0, ('q22','q23'):940.0, ('q22','q24'):1880.0, ('q23','q24'):3760.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q13 q14 q15 q16 q17 q18 q19 q2 ... q9  energy num_oc. ...
0    0   0   0   0   0   0   0   0   0   0   0  0 ...  0 -2463.0       1 ...
1    0   0   0   0   1   0   0   0   0   0   0  0 ...  0 -2412.0       4 ...
2    0   0   0   0   1   0   0   0   0   0   0  0 ...  0 -2340.0       2 ...
3    1   0   0   0   0   0   0   1   0   0   0  0 ...  0 -2320.0       1 ...
4    0   0   0   0   1   0   0   0   0   0   0  0 ...  0 -2320.0       1 ...
5    1   0   0   0   0   0   0   0   0   0   0  0 ...  0 -2289.0       1 ...
6    0   0   0   0   0   0   0   0   0   0   0  0 ...  0 -2289.0       1 ...
7    0   0   0   0   0   1   0   0   0   0   0  0 ...  0 -2289.0       2 ...
8    0   0   0   0   0   0   0   0   1   0   0  1 ...  0 -2279.0       1 ...
22   0   0   0   0   0   0   0   0   1   0   0  1 ...  0 -2279.0       3 ...
757  0   0   0   0   0   0   0   0   1   0   0  1 ...  0 -2279.0       1 ...
877  0   0   0   0   0   1   0   0   0   0   0  0 ...  0 -2279.0       1 ...